In [1]:
%pip install -q transformers
%pip install pandas
%pip install numpy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install torch --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\danie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import pipeline #hugging face pipeline
import os




# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Read the CSV files
x_test = pd.read_csv('./kaggle/input/desafioutfpr2024/X_testToronto.csv')
x_train = pd.read_csv('./kaggle/input/desafioutfpr2024/X_trainToronto.csv')
y_train = pd.read_csv('./kaggle/input/desafioutfpr2024/reviewsTrainToronto.csv')
y_test = pd.read_csv('./kaggle/input/desafioutfpr2024/reviewsTestToronto.csv')



x_train['destaque'].value_counts()




destaque
0    14184
1     3398
Name: count, dtype: int64

In [4]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce GTX 1060 with Max-Q Design


In [5]:

from transformers import AutoTokenizer, pipeline
import pickle

tokenizer = AutoTokenizer.from_pretrained("siebert/sentiment-roberta-large-english")
sentiment_pipeline = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english", tokenizer=tokenizer, device=0)

C:\Users\danie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:


data = y_train['text'].tolist()

# Define batch size
batch_size = 100

# Calculate number of batches
num_batches = len(data) // batch_size + (len(data) % batch_size != 0)

sentiment = []


In [ ]:

# Process data in batches
for i in range(3201, num_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, len(data))
    batch = data[start_index:end_index]

    
    # Truncate the tokens
    batch_sentiment = sentiment_pipeline(batch, max_length=512, truncation=True)
    sentiment.extend(batch_sentiment)

    # Save progress after each batch
    with open('./sentiment_batches/sentiment_batch_{}.pickle'.format(i), 'wb') as f:
        pickle.dump(batch_sentiment, f)

In [7]:
import pickle

# Initialize an empty list to store the sentiment batches
sentiment_batches = []

# Loop through the batch files and load each batch
for i in range(num_batches):
    with open('./sentiment_batches/sentiment_batch_{}.pickle'.format(i), 'rb') as f:
        batch = pickle.load(f)
        sentiment_batches.extend(batch)

# Now you have all the sentiment batches in a single object


In [10]:
# Create a new DataFrame with the required columns
df_export = pd.DataFrame({'business ID': y_train['business_id'],
                          'label': [row['score'] if row['label'] == 'POSITIVE' else row['score']*-1 for row in sentiment_batches]})

# Export the DataFrame to a CSV file
df_export.to_csv('exported_data.csv', index=False)


In [11]:
df_export_sum = df_export.groupby('business ID')['label'].sum().reset_index()
df_export_sum.to_csv('exported_review_sum.csv', index=False)
